<h2>ARC Liver-Spleen Project - CT CCHMC Datasheet Demographic Summary</h2>
Zachary Taylor

In [ ]:
!pip install openpyxl
import openpyxl
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
!pip install matplotlib-venn
import matplotlib_venn
from matplotlib_venn import venn3
import os
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [2]:
df = pd.read_excel("DemographicData_0207222.xlsx")
#df2 = pd.read_csv("")
#df = pd.concat(map(pd.read_csv, ['LS_all_features_batch1.csv', 'LS_all_features_batch2.csv']), ignore_index=True)

In [ ]:
dfsub = df[["ID", "gender", "age"]]
total = dfsub["ID"].count()
print("Total Patients / Samples:", total)
#dfsub.describe()

dfmask = pd.read_csv("cchmc_ct_standard_summary.csv")

i=0
while i<dfmask["ID"].count():
#dfmask["patid"] = dfmask["patid"].replace(dfmask["patid"].to_string(),dfmask["patid"].str.split('_',-1))
    dfmask["ID"].iat[i] = dfmask["ID"].iloc[i].split('_',-1)[-1]
    dfmask["ID"].iat[i]= int(dfmask["ID"].iloc[i])
    i=i+1
dfmasksub = dfmask[["ID", "spleen_volume"]]
print(dfmask["ID"])



#missingdirs = dft2["mask directory"].isnull().sum().sum()
#dft2['mask directory'] = dft2['mask directory'].fillna("missing")
#missingseqs = dfsubnew["sequence"].isnull().sum().sum()
#dft2['sequence'] = dft2['sequence'].fillna("missing")


#dft2sub = dft2[["Subject ID","T2 Directory", "mask directory", "sequence", "has directory"]]

dfsubnew = dfsub.merge(dfmasksub, how='outer', on='ID')
#dfsubnew = dfsub.merge(dft2sub, how='outer', on='Subject ID')
#dfsubnew['sequence'] = dfsubnew['sequence'].fillna("unknown")

print(dfsubnew.describe())
print("missing age: ",(total-dfsubnew['age'].count()))
print("missing gender: ",(total-dfsubnew['gender'].count()))
print("missing spleen mask: ",(total-dfsubnew['spleen_volume'].count()))


<h3>Categorical Analysis</h3>

In [ ]:
#subjectcount = dfsubnew["Subject ID"].value_counts()

#dft2['mask directory'] = dft2['mask directory'].fillna("missing")

#dfsubnew["has directory"] = dfsubnew["has directory"].fillna("unknown")
#dircount_dict = {1.0:"Found", 0.0:"Not Found", "unknown":"Unknown"}
#dircount = dfsubnew["has directory"].value_counts().rename(index=dircount_dict)
#percdir = dircount / dircount.sum()

#print("\nMask Directory Count:")
#print("\nmissing directories:",missingdirs)

#t2 = dft2sub["Subject ID"].count() - missingperc
#found = {"T2":[t2],"Missing Directory":[missingdirs]}
#dft2found = pd.DataFrame(data=found)
#print(pd.concat([dircount,percdir], axis=1,keys=['counts', '%']))

print('\nMasking:')

spleencount = dfsubnew["spleen_volume"].count()
spleenmissing = abs(total - spleencount)
percspleen = [spleencount / total , spleenmissing / total]

arrspleen = np.array([spleencount, spleenmissing])
arrspleenlabels = ['Spleen Mask Found', 'Spleen Mask Missing']
print("\n Spleen")
print(pd.concat([pd.DataFrame(arrspleenlabels),pd.DataFrame(arrspleen),pd.DataFrame(percspleen)], axis=1,keys=['','counts', '%']))

print('\nGender:')
gendercount = dfsub["gender"].value_counts(dropna=False)
genderperc = dfsub["gender"].value_counts(dropna=False, normalize=True)
print(pd.concat([gendercount,genderperc], axis=1,keys=['counts', '%']))

<h3>Discrete Analysis<h3>

In [ ]:
print('Age:\n')

i=0
while i<dfsubnew["age"].count():
#dfmask["patid"] = dfmask["patid"].replace(dfmask["patid"].to_string(),dfmask["patid"].str.split('_',-1))
    #print(dfsubnew["age"].iat[i])
    dfsubnew["age"].iat[i] = dfsubnew["age"].iloc[i].split('y, ')
    #print(dfsubnew["age"].iat[i])
    dfsubnew["age"].iat[i] = str(int(dfsubnew["age"].iloc[i][0]))+'.'+(str(int((float(dfsubnew["age"].iloc[i][1].replace('m',''))/12)*10)))
    #print(dfsubnew["age"].iat[i])
    dfsubnew["age"].iat[i]= float(dfsubnew["age"].iloc[i])
    i=i+1

print(dfsubnew.agg({"age":["min","max","mean","median","skew"]}))

#nullheight = heightdata.isnull().sum().sum()
#nullweight = weightdata.isnull().sum().sum()
#hnullperc = nullheight / heightdata.size * 100
#wnullperc = nullweight / weightdata.size * 100
#print(dfsub.agg({"HEIGHT":["min","max","mean","median","skew"], "WEIGHT":["min","max","mean","median","skew"]}))
#print("NOTE:",nullheight," patients' height not listed; ",nullweight," patients' weight not listed\n",hnullperc,"% patients missing height data\n",wnullperc,"% patients missing weight data")

<h3>Figures and Graphs<h3>

In [ ]:
print('Categorical:\n')
figt, axt = plt.subplots(1,2, figsize = (15,5))

colorsgen = ['#c7846b','#6b70c7']
axt[0].pie(gendercount, shadow=True, labels=gendercount.index, autopct='%1.1f%%', startangle=90)

axt[0].set_title('Gender ('+str(total)+' Samples)')

colorssp = ['#784880','#FAC9B8']
axt[1].pie(arrspleen, labels=arrspleenlabels,colors=colorssp,autopct='%1.1f%%',
        shadow=True, startangle=90)
axt[1].set_title('Spleen Masks Found ('+str(total)+' Samples)')
plt.show()


print('\n\nDiscrete:\n')

agedata = dfsubnew["age"]

bins = [0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0]
plt.figure(figsize=(9,5))
plt.suptitle('Age', fontsize=14)
plt.hist(dfsubnew["age"], bins=bins, edgecolor="red")
plt.xlabel('Age (years)')
plt.ylabel('Count')
plt.axis([0, 20, 0, 25])
plt.grid()
plt.show()
print('\n\n')

plt.show()